In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import os
import sys
sys.path.append(".bin")

DATA_PATH = "data"
CODE_CONTEST_DATA_PATH = f"code_contests/data/code_contest_data.jsonl"
CODE_CONTEST_DATA_PATH = f"{DATA_PATH}/code_contest_data.jsonl"
DOMAIN_DATA_PATH = f"{DATA_PATH}/domain_data.jsonl"
os.makedirs(DATA_PATH, exist_ok=True)

In [3]:
from domain.domain_dao import DomainFileDAO
from domain.problems_d import PromptedSolutionD, TestResultD, ContestProblemD



In [8]:
import fastparquet as fp
import glob

problems = []
parquet_files = glob.glob("code_contests/data/train*.parquet")
for parquet_file in parquet_files:
    df = fp.ParquetFile(parquet_file).to_pandas()
    problems += ContestProblemD.from_df(df)


AttributeError: 'NoneType' object has no attribute 'extend'

In [ ]:
print(*proto_dicts[0].keys(), sep="\n")

In [5]:
test_dicts = ContestProblemD.from_df(df)

In [7]:
sum(len(d.incorrect_solutions) for d in test_dicts)

7236

In [ ]:
print(test_dicts[0].to_json())